In [171]:
import numpy as np
from numpy.linalg import norm, cholesky, solve
from numpy import matmul


In [172]:
def generate_householder_mat(dim):
    rng = np.random.default_rng(52)

    w = rng.random(dim)
    w = w / norm(w, ord=None) 

    lambdas = rng.random(dim)
    lambda_mat = np.eye(dim)

    for i in range(dim):
        lambda_mat[i, i] = lambdas[i]
    print(lambdas, "\n", lambda_mat)

    H = np.eye(dim) - 2 * matmul(w.reshape(dim, -1), w.reshape(-1, dim))
    # print(w.reshape(dim, -1), w.reshape(-1, dim))
    print(H)

    return matmul(matmul(H, lambda_mat), H.T)


size_H = 10
test_hh = generate_householder_mat(size_H)
print(test_hh)


[0.89601131 0.47378689 0.11715695 0.07880772 0.98278754 0.06445595
 0.48948325 0.40900674 0.76510318 0.07251445] 
 [[0.89601131 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.47378689 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.11715695 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.07880772 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.98278754 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.06445595
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.48948325 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.40900674 0.         0.        ]
 [0.         0.      

In [173]:
# def norm(vec):
#   sum = 0
#   for i in vec:
#     sum += i*i 
#   return sum**(1/2)

In [174]:
def generate_positive_definite_matrix(size: int) -> np.ndarray:
    """
    Generate random symmetric positive definite matrix:
    1. Create random matrix A
    2. Compute A^T * A (always positive semi-definite)
    3. Add diagonal dominance to ensure positive definite
    """
    rng = np.random.default_rng(52)
    
    # Create random matrix
    A = rng.random((size, size))
    
    # Make symmetric positive definite
    matrix = A @ A.T  # Ensures positive semi-definite
    
    # Add to diagonal to ensure positive definite
    matrix += size//2 * np.eye(size)
    
    return matrix

In [175]:
# mat_size = 3
# mat = generate_positive_definite_matrix(3)

# print(mat)
# mat = np.ndarray((2, 2), dtype=np.int64)


def inverse_iteration(A: np.ndarray, tolerance: np.float64 = 1e-13, max_iter: int = 500):
    dim = A.shape[0]

    # initial guess
    x_prev = np.random.rand(dim)
    # print(x_prev)

    sigma = 0
    sigma_prev = 0

    # decompose A into L*U using cholesky
    L = cholesky(A, upper=False)
    U = L.T

    A_inv = np.linalg.inv(A)

    # U = np.linalg.cholesky(A, upper=True)

    # print(L @ L.T) # A matr

    for _ in range(max_iter):
        x_norm = norm(x_prev, ord=None)
        # print(norm)
        nu = x_prev / x_norm  # +- eigenvec

        # L*y = nu
        # U*x = y
        y = solve(L, nu)
        x_next = solve(U, y)
        # x_next = matmul(A_inv, nu)

        sigma_prev = sigma
        sigma = matmul(nu, x_next)  # eigenvalue

        if abs(sigma - sigma_prev) < tolerance and abs(sigma - sigma_prev) != 0:
            # print(_)
            return nu, 1 / sigma

        sigma_prev = sigma
        x_prev = x_next

    return nu, 1 / sigma


eigvec_1, eigval_1 = inverse_iteration(test_hh)

print(eigval_1, eigvec_1)


0.06445594702461907 [ 0.35776894  0.33363542  0.30186382  0.00371243  0.31972941 -0.4373786
  0.23127958  0.00920181  0.14162437  0.54990356]


In [190]:
def inverse_iteration_with_shift_one(
    A: np.ndarray, tolerance: np.float64 = 1e-14, max_iter: int = 500
):
    dim = A.shape[0]

    # initial guess
    x_prev = np.random.rand(dim)
    # print(x_prev)

    sigma = 0
    sigma_prev = 0

    # decompose A into L*U using cholesky
    L = cholesky(A, upper=False)
    U = L.T

    g_1, _ = inverse_iteration(A)
    print(g_1, _)

    # U = np.linalg.cholesky(A, upper=True)

    # print(np.matmul(g_1.reshape(dim, -1), g_1.reshape(-1, dim)))
    eyeminus = np.eye(dim) - np.matmul(g_1.reshape(dim, -1), g_1.reshape(-1, dim))

    # print(L @ L.T) # A matr

    for _ in range(max_iter):
        x_norm = norm(x_prev, ord=None)
        # print(norm)
        nu = x_prev / x_norm  # +- eigenvec

        # f = E - g_1*g_1^T
        # L*y = f * nu
        # U*x = y

        y = solve(
            L,
            matmul(eyeminus, nu),
        )
        x_next = solve(U, y)

        # print(g_1.reshape(dim, -1).shape, g_1.reshape(-1, dim).shape)
        # x_next = matmul(
        #     matmul(
        #         np.linalg.inv(A),
        #         np.eye(dim) - np.matmul(g_1.reshape(dim, -1), g_1.reshape(-1, dim)),
        #     ),
        #     nu,
        # )

        sigma_prev = sigma
        sigma = matmul(nu, x_next)  # eigenvalue

        if abs(sigma - sigma_prev) < tolerance:
          # print(_)
          return nu, 1 / sigma

        sigma_prev = sigma
        x_prev = x_next

    return nu, 1 / sigma


In [177]:
print(inverse_iteration_with_shift_one(test_hh))

[ 0.35776894  0.33363542  0.30186382  0.00371243  0.31972941 -0.4373786
  0.23127958  0.00920181  0.14162437  0.54990356] 0.06445594702461904
183
(array([ 0.34968166,  0.32609368,  0.29504026,  0.00362875,  0.31250201,
        0.54990399,  0.22605156,  0.0089938 ,  0.13842299, -0.46252643]), np.float64(0.07251445258539307))


In [178]:
print(np.linalg.eig(test_hh))

EigResult(eigenvalues=array([0.98278754, 0.89601131, 0.76510318, 0.48948325, 0.47378689,
       0.40900674, 0.11715695, 0.06445595, 0.07251445, 0.07880772]), eigenvectors=array([[ 2.03314702e-01, -7.72496107e-01,  9.00583941e-02,
        -1.47069793e-01, -2.12157482e-01, -5.85139311e-03,
         1.91954040e-01,  3.57768783e-01, -3.49681846e-01,
         2.36072267e-03],
       [ 1.89599987e-01,  2.12157482e-01,  8.39834511e-02,
        -1.37149112e-01,  8.02153729e-01, -5.45668388e-03,
         1.79005665e-01,  3.33635275e-01, -3.26093848e-01,
         2.20147871e-03],
       [ 1.71544662e-01,  1.91954040e-01,  7.59858319e-02,
        -1.24088606e-01, -1.79005665e-01, -4.93705199e-03,
        -8.38040778e-01,  3.01863684e-01, -2.95040416e-01,
         1.99183516e-03],
       [ 2.10972050e-03,  2.36072267e-03,  9.34502217e-04,
        -1.52608815e-03, -2.20147871e-03, -6.07177142e-05,
         1.99183516e-03,  3.71243263e-03, -3.62851752e-03,
        -9.99975504e-01],
       [-8.183025

## Testing

In [197]:
def TEST_generate_householder_mat(dim, lambda_range):
    rng = np.random.default_rng()

    w = rng.uniform(-lambda_range, lambda_range, dim)
    w = rng.random(dim)
    w = w / norm(w, ord=None) 

    lambdas = rng.uniform(-lambda_range, lambda_range, dim)
    lambdas = rng.random(dim)
    lambda_mat = np.eye(dim)

    for i in range(dim):
        lambda_mat[i, i] = lambdas[i]
    # print(lambdas, "\n", lambda_mat)

    # second min
    second_lam = np.sort(lambdas)[1]
    # print(second_lam)
    H = np.eye(dim) - 2 * matmul(w.reshape(dim, -1), w.reshape(-1, dim))
    second_g = H[:, 1]
    # print(second_g)
    print(H)

    return matmul(matmul(H, lambda_mat), H.T)

def TEST(size_range, lambda_range, lambda_diff=10**(-5)):
  rng = np.random.default_rng()
  cnt = 0

  lambdas_diffs = []
  vectors_diffs = []

  while cnt < 10:
    
    size = rng.integers(2, 4)
    # A = TEST_generate_householder_mat(size, lambda_range)
    try:
      A = TEST_generate_householder_mat(size, lambda_range)
      vec, val = inverse_iteration_with_shift_one(A)
      print(vec, val)
      cnt+=1
      print("")
    except np.linalg.LinAlgError:
      print("except")
  
  
  # print(size_range, " -- ", lambda_range, " -- ", "eps=10**(-13)", lambdas_mean, vectors_mean, r_val, iterat_mean)



TEST(10, 2)
  

[[ 0.30815981 -0.87368846 -0.37643859]
 [-0.87368846 -0.10333503 -0.47538442]
 [-0.37643859 -0.47538442  0.79517523]]
[-0.3081598   0.87368846  0.37643859] 0.10906275835010237
[ 0.37643872  0.47538442 -0.79517517] 0.6294269699418541

[[-0.96612475 -0.25807549]
 [-0.25807549  0.96612475]]
[0.9661247  0.25807568] 0.5038337758886929
[-0.25807585  0.96612466] 0.9493115894104638

[[-0.16714846 -0.82610802 -0.5381514 ]
 [-0.82610802  0.4152805  -0.38090372]
 [-0.5381514  -0.38090372  0.75186796]]
[0.16714851 0.82610805 0.53815134] 0.2528668529180067
[-0.53815125 -0.38090363  0.75186811] 0.5207776810703052

[[-0.43335474 -0.90122343]
 [-0.90122343  0.43335474]]
[0.43335539 0.90122312] 0.3465148422405878
[ 0.90122307 -0.4333555 ] 0.4048012796739887

[[ 0.98423412 -0.17264802 -0.03841683]
 [-0.17264802 -0.89062355 -0.4206927 ]
 [-0.03841683 -0.4206927   0.90638943]]
[-0.03841656 -0.42069129  0.90639009] 0.36431762058935724
[0.1726483  0.89062423 0.42069114] 0.39389574141579936

[[ 0.59002443 -0